In [140]:
import os
import json
from deep_translator import GoogleTranslator
#from deep_translator.exceptions import NotValidPayload

In [141]:
data_path = '../data/2023-04-12_oasst_ready.trees.jsonl'
with open(data_path, 'r') as f:
    data = [json.loads(line) for line in f]

In [147]:
len(data) // 2

5182

### Translate the data

In [113]:
translator = GoogleTranslator(source='en', target='sl')
#translator = PonsTranslator(source='en', target='sl')
#translator = DeeplTranslator(source='en', target='sl')
#translator = MyMemoryTranslator(source='en', target='sl')

def _translate(text):
    return translator.translate(text, from_language='sl')


def translate(text):
    if len(text) < 5000: 
        return _translate(text)

    lines = text.split('\n')
    translations = []
    for line in lines:
        translations.append(_translate(line))
    return '\n'.join(translations)

In [121]:
def translate_message(message, translate):
    text = message['text']
    translation = translate(text)
    if len(translation) == 0:
        print('Translation failed for:')
        print(text)
    message['translation'] = translation
    for reply in message['replies']:
        reply = translate_message(reply, translate)
    return message
    

In [122]:
def translate_tree(tree, translate):
    tree['prompt'] = translate_message(tree['prompt'], translate)
    return tree

In [123]:
def translate_dataset(data, translate, translations_path='data/translated'):
    #os.mkdir(translations_path)
    for tree in data:
        if tree['prompt']['lang'] != 'en':
            continue
        # catch errors at this stage so as not to stop translation of other trees
        try:
            translated = translate_tree(tree, translate)
            # save to file
            tree_path = os.path.join(translations_path, tree['message_tree_id'] + '.json')
            with open(tree_path, 'w', encoding='utf-8') as f:
                json.dump(translated, f, ensure_ascii=False)
        except Exception as e:
            print(e)
            continue

In [124]:
translate_dataset(data, translate, translations_path='../data/google_translate')

In [139]:
len(data)

66497

In [138]:
5 * len(data) / 60 / 60

92.35694444444445

## Check the transalted conversation

In [132]:
def generate_conversation(tree):
    conversation = []
    message = tree['prompt']
    conversation.append((message['role'], message['translation']))
    while len(message['replies']) > 0:
        message = message['replies'][0]
        conversation.append((message['role'], message['translation']))
    return conversation

In [133]:
tre_id = data[2]['message_tree_id']

with open('../data/google_translate/' + tre_id + '.json', 'r') as f:
    translated = json.load(f)

In [134]:
conv = generate_conversation(translated)
for role, line in conv:
    print(f'{role.upper()}: {line}')

PROMPTER: Ali lahko kontrastno učenje v strojnem učenju preprosto razložite nekomu, ki je nov na področju strojnega učenja?
ASSISTANT: Seveda! Recimo, da želite zgraditi model, ki lahko razlikuje med slikami mačk in psov. Zberete svoj nabor podatkov, sestavljen iz številnih slik mačk in psov. Nato jih prenesete skozi nevronsko mrežo po vaši izbiri, ki ustvari nekaj predstavitve za vsako sliko, zaporedje številk, kot je [0,123, 0,045, 0,334, ...]. Težava je v tem, da če vaš model ne pozna podob mačke in psa, bodo te predstavitve precej naključne. Včasih bi lahko imeli sliki mačke in psa zelo podobni predstavitvi (njuni številki bi bili blizu ena drugi), medtem ko sta lahko drugič dve podobi mačke predstavljeni daleč narazen. Preprosto povedano, model ne bi mogel ločiti mačk od psov. Tu nastopi kontrastno učenje.

Bistvo kontrastivnega učenja je, da vzamemo pare vzorcev (v tem primeru slike mačk in psov), nato urimo model, da "povleče" predstavitve podobnih parov (mačka-mačka ali pes-pes